In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os, gc
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_rows",200)
pd.set_option("display.max_columns",100)


In [2]:
train_df = pd.read_csv('train_aox2Jxw/train.csv')
test_df  = pd.read_csv('test_bqCt9Pv.csv')
sub      = pd.read_csv('sample_submission_24jSKY6.csv')
train_test = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

train_test.shape, train_df.shape, test_df.shape, sub.shape

((345546, 41), (233154, 41), (112392, 40), (112392, 2))

# Only `Employment.Type` have `NaN` values

## Add flag to diffrentiate b/w train and test

In [3]:
def get_quantile(df, col, q1, q2):
    lower_bound = np.percentile(df[col], q=q1)
    upper_bound = np.percentile(df[col], q=q2)
    print("low: ", lower_bound)
    print("high: ", upper_bound)
    

# Handling `12` catgorical variables

In [4]:
train_test['train_flag'] = 0
train_test['train_flag'].iloc[:train_df.shape[0]] = 1
train_test.train_flag.value_counts()

1    233154
0    112392
Name: train_flag, dtype: int64

In [5]:
train_test['Date.of.Birth'] = train_test['Date.of.Birth'].apply(
    lambda x: str(x)[:-2]+'20'+str(x)[-2:] if str(x)[-2:] == '00' \
    else str(x)[:-2]+'19'+str(x)[-2:])
train_test['date_of_birth']  = pd.to_datetime(pd.Series(train_test['Date.of.Birth']))
train_test.drop('Date.of.Birth', axis=1, inplace=True)

In [6]:
train_test['DisbursalDate_new'] = pd.to_datetime(
    pd.Series(train_test['DisbursalDate'].apply(lambda x: str(x)[:-3])), 
    format="%d-%m")

train_test.DisbursalDate_new = train_test.DisbursalDate_new.apply(
    lambda dt: dt.replace(year=2018))

train_test['disbursal_week']  = train_test['DisbursalDate_new'].dt.week
train_test['disbursal_day']   = train_test['DisbursalDate_new'].dt.day
train_test['disbursal_month'] = train_test['DisbursalDate_new'].dt.month

# train_test.DisbursalDate_new.unique()

In [7]:
train_test['age(in years)'] = ((train_test['DisbursalDate_new'] - \
                    train_test['date_of_birth'])/365) / np.timedelta64(1, 'D')
train_test['age(in years)'] = train_test['age(in years)'].astype('int')
print(train_test['age(in years)'][:5])

train_test['age(in month)'] = ((train_test['DisbursalDate_new'] - \
                    train_test['date_of_birth'])/30) / np.timedelta64(1, 'D')
train_test['age(in month)'] = train_test['age(in years)'].astype('int')
print(train_test['age(in month)'][:5])

0    34
1    33
2    32
3    24
4    41
Name: age(in years), dtype: int64
0    34
1    33
2    32
3    24
4    41
Name: age(in month), dtype: int64


In [8]:
train_test['credit_hist_year'] = train_test['CREDIT.HISTORY.LENGTH'].apply(lambda x: x.split(' ')[0][:-3])
train_test['credit_hist_month']= train_test['CREDIT.HISTORY.LENGTH'].apply(lambda x: x.split(' ')[1][:-3])

train_test['credit_hist_year'] = train_test['credit_hist_year'].astype('int')
train_test['credit_hist_month']= train_test['credit_hist_month'].astype('int')

train_test['credit_hist_total_month']= train_test['credit_hist_month'] + \
                                        train_test['credit_hist_year']*12

train_test['loan_tenure_year'] = train_test['AVERAGE.ACCT.AGE'].apply(lambda x: x.split(' ')[0][:-3])
train_test['loan_tenure_month']= train_test['AVERAGE.ACCT.AGE'].apply(lambda x: x.split(' ')[1][:-3])

train_test['loan_tenure_year'] = train_test['loan_tenure_year'].astype('int')
train_test['loan_tenure_month']= train_test['loan_tenure_month'].astype('int')

train_test['loan_tenure_total_month']= train_test['loan_tenure_month'] + \
                                        train_test['loan_tenure_year']*12

# train_test.drop(['AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH'], axis=1, inplace=True)


In [9]:
train_test['Employment.Type'].fillna('Self employed', inplace=True)


In [10]:
col = 'PRI.NO.OF.ACCTS'
new_col = 'pri_no_of_accts'

bins = pd.IntervalIndex.from_tuples([(-1,0),(0,1),(1,2),(2,3),(3,4),(4,8),(8,15),(15,1000)])
train_test[new_col] = pd.cut(train_test[col], bins)
print(train_test[new_col].value_counts())

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_test[new_col] = le.fit_transform(train_test[new_col])
print(le.classes_)

train_test.drop(col, axis=1, inplace=True)
train_test.groupby([new_col,'loan_default'])['loan_default'].count().unstack()


(-1, 0]       174843
(0, 1]         51914
(4, 8]         30184
(1, 2]         29253
(2, 3]         19409
(8, 15]        17566
(3, 4]         13758
(15, 1000]      8619
Name: pri_no_of_accts, dtype: int64
[Interval(-1, 0, closed='right') Interval(0, 1, closed='right')
 Interval(1, 2, closed='right') Interval(2, 3, closed='right')
 Interval(3, 4, closed='right') Interval(4, 8, closed='right')
 Interval(8, 15, closed='right') Interval(15, 1000, closed='right')]


loan_default,0.0,1.0
pri_no_of_accts,,
0,89898,27052
1,27241,7737
2,15535,4249
3,10377,2638
4,7535,1788
5,16791,3969
6,10026,2176
7,5140,1002


In [11]:
col = 'PRI.ACTIVE.ACCTS'

bins = pd.IntervalIndex.from_tuples([(-1, 0),(0, 1),(1,3),(3,6),(6,10),(10,500)])
train_test['no_of_acc'] = pd.cut(train_test[col], bins)
print(train_test['no_of_acc'].value_counts())

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_test['no_of_acc'] = le.fit_transform(train_test['no_of_acc'])
print(le.classes_)

train_test.drop(col, axis=1, inplace=True)
train_test.groupby(['no_of_acc','loan_default'])['loan_default'].count().unstack()

(-1, 0]      205427
(0, 1]        62095
(1, 3]        49528
(3, 6]        21052
(6, 10]        5835
(10, 500]      1609
Name: no_of_acc, dtype: int64
[Interval(-1, 0, closed='right') Interval(0, 1, closed='right')
 Interval(1, 3, closed='right') Interval(3, 6, closed='right')
 Interval(6, 10, closed='right') Interval(10, 500, closed='right')]


loan_default,0.0,1.0
no_of_acc,,
0,105965,31051
1,32873,9182
2,26821,6996
3,12173,2617
4,3659,609
5,1052,156


In [12]:
col = 'PRI.OVERDUE.ACCTS'
new_col = 'no_of_acc_overdue'
bins = pd.IntervalIndex.from_tuples([(-1, 0),(0, 1),(1,2),(2,5),(5,500)])
train_test[new_col] = pd.cut(train_test[col], bins)
print(train_test[new_col].value_counts())

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_test[new_col] = le.fit_transform(train_test[new_col])
print(le.classes_)
print(train_test.groupby([new_col,'loan_default'])['loan_default'].count().unstack())

train_test.drop(col, axis=1, inplace=True)


(-1, 0]     306917
(0, 1]       29378
(1, 2]        6275
(2, 5]        2642
(5, 500]       334
Name: no_of_acc_overdue, dtype: int64
[Interval(-1, 0, closed='right') Interval(0, 1, closed='right')
 Interval(1, 2, closed='right') Interval(2, 5, closed='right')
 Interval(5, 500, closed='right')]
loan_default          0.0    1.0
no_of_acc_overdue               
0                  163471  43408
1                   14588   5382
2                    3061   1241
3                    1243    529
4                     180     51


In [13]:
train_test.drop('MobileNo_Avl_Flag', axis=1, inplace=True)


In [14]:
# bureau description and corresponding score
mapping = {
        'No Bureau History Available'                            :'not_enough_info',# 00
        'Not Scored: More than 50 active Accounts found'         :'not_enough_info',# 11
        'Not Scored: No Activity seen on the customer (Inactive)':'not_enough_info',# 16
        'Not Scored: No Updates available in last 36 months'     :'not_enough_info',# 18 
        'Not Scored: Not Enough Info available on the customer'  :'not_enough_info',# 17
        'Not Scored: Only a Guarantor'                           :'not_enough_info',# 14
        'Not Scored: Sufficient History Not Available'           :'not_enough_info' # 15
       }

train_test['Bureau_desc'] = train_test['PERFORM_CNS.SCORE.DESCRIPTION'].replace(mapping)

use_index = train_test[['PERFORM_CNS.SCORE.DESCRIPTION','PERFORM_CNS.SCORE']][
    train_test['Bureau_desc'] == 'not_enough_info']['PERFORM_CNS.SCORE'].index
train_test['bureau_score'] = train_test['PERFORM_CNS.SCORE']
train_test.loc[use_index,'bureau_score'] = 0


train_test.drop(['PERFORM_CNS.SCORE.DESCRIPTION','PERFORM_CNS.SCORE'], 
                axis=1, inplace=True)


In [15]:
col = 'NEW.ACCTS.IN.LAST.SIX.MONTHS'
new_col = 'new_acc_past_month'

bins = pd.IntervalIndex.from_tuples([(-1,0),(0,1),(1,2),(2,3),(3,4),(4,50)])
train_test[new_col] = pd.cut(train_test[col], bins)
print(train_test[new_col].value_counts())

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_test[new_col] = le.fit_transform(train_test[new_col])
print(le.classes_)

train_test.groupby([new_col,'loan_default'])['loan_default'].count().unstack()
train_test.drop(col, axis=1, inplace=True)

(-1, 0]    272341
(0, 1]      45965
(1, 2]      15618
(2, 3]       6161
(4, 50]      2825
(3, 4]       2636
Name: new_acc_past_month, dtype: int64
[Interval(-1, 0, closed='right') Interval(0, 1, closed='right')
 Interval(1, 2, closed='right') Interval(2, 3, closed='right')
 Interval(3, 4, closed='right') Interval(4, 50, closed='right')]


# numeric columns

In [16]:
def print_it(col):
    print(col)
    print("train-test:")
    get_quantile(train_test,col,q1,q2)
    print("train_df  :")
    get_quantile(train_df,col,q1,q2)
    print("*"*30)

q1 = 1
q2 = 99

col = 'PRI.SANCTIONED.AMOUNT'; print_it(col)
col = 'PRI.DISBURSED.AMOUNT'; print_it(col)
col = 'PRI.SANCTIONED.AMOUNT'; print_it(col)
col = 'PRI.DISBURSED.AMOUNT'; print_it(col)
col = 'PRI.CURRENT.BALANCE'; print_it(col)
col = 'SEC.CURRENT.BALANCE'; print_it(col)
col = 'SEC.DISBURSED.AMOUNT'; print_it(col)
col = 'PRIMARY.INSTAL.AMT'; print_it(col)
col = 'SEC.CURRENT.BALANCE'; print_it(col)



PRI.SANCTIONED.AMOUNT
train-test:
low:  0.0
high:  3363234.2999999993
train_df  :
low:  0.0
high:  3467119.3700000034
******************************
PRI.DISBURSED.AMOUNT
train-test:
low:  0.0
high:  3366855.2999999924
train_df  :
low:  0.0
high:  3457573.880000001
******************************
PRI.SANCTIONED.AMOUNT
train-test:
low:  0.0
high:  3363234.2999999993
train_df  :
low:  0.0
high:  3467119.3700000034
******************************
PRI.DISBURSED.AMOUNT
train-test:
low:  0.0
high:  3366855.2999999924
train_df  :
low:  0.0
high:  3457573.880000001
******************************
PRI.CURRENT.BALANCE
train-test:
low:  0.0
high:  2835998.64999999
train_df  :
low:  0.0
high:  2917864.330000001
******************************
SEC.CURRENT.BALANCE
train-test:
low:  0.0
high:  6226.499999999884
train_df  :
low:  0.0
high:  19733.780000000086
******************************
SEC.DISBURSED.AMOUNT
train-test:
low:  0.0
high:  30397.29999999993
train_df  :
low:  0.0
high:  47544.83000000022
***

In [17]:
train_test.shape, train_df.shape, test_df.shape, train_df.shape[0]+test_df.shape[0]

((345546, 53), (233154, 41), (112392, 40), 345546)

In [18]:
def replace_outliers(df, low, high):
    """
    Args:
        df  : array
        low : 01% range
        high: 99% range
    return:
        an array, where outliers are replaced by the end ranges
    """
    df = np.where(df>high, high, df)
    df = np.where(df<low, low, df)
    print(df)
    
x = [1,2,3,4,54,5,7,78,9,9,33,5,6,9]
replace_outliers(np.array(x), 3,7)

[3 3 3 4 7 5 7 7 7 7 7 5 6 7]


In [19]:
def count_negative(df, col):
    print(col, "==", df[df[col]<0].shape[0])
print("count negative numbers")

col = 'PRI.SANCTIONED.AMOUNT'; count_negative(train_test, col)
col = 'PRI.DISBURSED.AMOUNT'; count_negative(train_test, col)
col = 'PRI.SANCTIONED.AMOUNT'; count_negative(train_test, col)
col = 'PRI.DISBURSED.AMOUNT'; count_negative(train_test, col)
col = 'PRI.CURRENT.BALANCE'; count_negative(train_test, col)
col = 'SEC.CURRENT.BALANCE'; count_negative(train_test, col)
col = 'SEC.DISBURSED.AMOUNT'; count_negative(train_test, col)
col = 'PRIMARY.INSTAL.AMT'; count_negative(train_test, col)
col = 'SEC.CURRENT.BALANCE'; count_negative(train_test, col)



count negative numbers
PRI.SANCTIONED.AMOUNT == 1
PRI.DISBURSED.AMOUNT == 0
PRI.SANCTIONED.AMOUNT == 1
PRI.DISBURSED.AMOUNT == 0
PRI.CURRENT.BALANCE == 666
SEC.CURRENT.BALANCE == 66
SEC.DISBURSED.AMOUNT == 0
PRIMARY.INSTAL.AMT == 0
SEC.CURRENT.BALANCE == 66


In [20]:
def print_it(col):
    print(col)
#     print("train-test:")
    get_quantile(train_test,col,q1,q2)
#     print("train_df  :")
#     get_quantile(train_df,col,q1,q2)
    print("*"*30)

q1 = 1
q2 = 99

col = 'PRI.SANCTIONED.AMOUNT'; print_it(col)
col = 'PRI.DISBURSED.AMOUNT'; print_it(col)
col = 'PRI.SANCTIONED.AMOUNT'; print_it(col)
col = 'PRI.DISBURSED.AMOUNT'; print_it(col)
col = 'PRI.CURRENT.BALANCE'; print_it(col)
col = 'SEC.CURRENT.BALANCE'; print_it(col)
col = 'SEC.DISBURSED.AMOUNT'; print_it(col)
col = 'PRIMARY.INSTAL.AMT'; print_it(col)
col = 'SEC.CURRENT.BALANCE'; print_it(col)

PRI.SANCTIONED.AMOUNT
low:  0.0
high:  3363234.2999999993
******************************
PRI.DISBURSED.AMOUNT
low:  0.0
high:  3366855.2999999924
******************************
PRI.SANCTIONED.AMOUNT
low:  0.0
high:  3363234.2999999993
******************************
PRI.DISBURSED.AMOUNT
low:  0.0
high:  3366855.2999999924
******************************
PRI.CURRENT.BALANCE
low:  0.0
high:  2835998.64999999
******************************
SEC.CURRENT.BALANCE
low:  0.0
high:  6226.499999999884
******************************
SEC.DISBURSED.AMOUNT
low:  0.0
high:  30397.29999999993
******************************
PRIMARY.INSTAL.AMT
low:  0.0
high:  240000.0
******************************
SEC.CURRENT.BALANCE
low:  0.0
high:  6226.499999999884
******************************


In [21]:
col = 'PRI.SANCTIONED.AMOUNT'

print(train_test.shape, "==>", end=" ")
train_test[train_test[col] < 0][col] = 0
train_test[train_test[col] > 3363234][col] = 3363234
print(train_test.shape)

col = 'PRI.DISBURSED.AMOUNT'

print(train_test.shape, "==>", end=" ")
train_test[train_test[col] < 0][col] = 0
train_test[train_test[col] > 3366855][col] = 3366855
print(train_test.shape)

col = 'PRI.CURRENT.BALANCE'

print(train_test.shape, "==>", end=" ")
train_test[train_test[col]<0][col] = 0
train_test[train_test[col]>2835998][col] = 2835998
print(train_test.shape)

col = 'SEC.CURRENT.BALANCE'

print(train_test.shape, "==>", end=" ")
train_test[train_test[col]<0][col] = 0
train_test[train_test[col]>6226][col] = 6226
print(train_test.shape)

col = 'SEC.DISBURSED.AMOUNT'

print(train_test.shape, "==>", end=" ")
train_test[train_test[col]<0][col] = 0
train_test[train_test[col]>30397][col] = 30397
print(train_test.shape)

col = 'SEC.CURRENT.BALANCE'

print(train_test.shape, "==>", end=" ")
train_test[train_test[col]<0][col] = 0 
train_test[train_test[col]>6226.5][col] = 6226.5
print(train_test.shape)

col = 'PRIMARY.INSTAL.AMT'

print(train_test.shape, "==>", end=" ")
train_test[train_test[col]<0][col] = 0
train_test[train_test[col]>240000][col] = 240000
print(train_test.shape)


(345546, 53) ==> (345546, 53)
(345546, 53) ==> (345546, 53)
(345546, 53) ==> (345546, 53)
(345546, 53) ==> (345546, 53)
(345546, 53) ==> (345546, 53)
(345546, 53) ==> (345546, 53)
(345546, 53) ==> (345546, 53)


In [22]:
col1 = 'PRI.SANCTIONED.AMOUNT'
col2 = 'PRI.DISBURSED.AMOUNT'
train_test['pay_by_yourself'] = train_test[col2] - train_test[col1]

new_col1 = 'obtained_amount_per_month'
new_col2 = 'obtained_amount_per_year'
train_test[new_col1] = train_test[col2]/(1 + train_test['loan_tenure_total_month'])
train_test[new_col2] = train_test[col2]/(1 + train_test['loan_tenure_year'])


In [23]:
print(train_test.shape[0], "==>", end=" ")
drop_cols = ['SEC.OVERDUE.ACCTS', 'SEC.ACTIVE.ACCTS', 'SEC.NO.OF.ACCTS', 'VoterID_flag', 
             'Driving_flag', 'Passport_flag', 'SEC.INSTAL.AMT']
train_test.drop(drop_cols, axis=1, inplace=True)
print(train_test.shape[0])

345546 ==> 345546


In [24]:
############## Simple aggregation w/o time compomnenet###########
print(train_test.shape, "==>", end=" ")

####################### manufacturer_id Mean #######################
branch_gps = train_test.groupby(['manufacturer_id'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['manufac_disbursed_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['manufacturer_id'],how='left')

####################### State_ID Mean #######################
branch_gps = train_test.groupby(['State_ID'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['state_disbursed_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['State_ID'],how='left')

####################### branch_id Mean #######################
branch_gps = train_test.groupby(['branch_id'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['branch_disbursed_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['branch_id'],how='left')

print(train_test.shape)


(345546, 49) ==> (345546, 52)


# Aggregation with time compomnenet
    - It have different distribution for train and test

In [25]:
############## Aggregation with time compomnenet ###########

print(train_test.shape, "==>", end=" ")

####################### manufacturer_id Mean #######################
branch_gps = train_test.groupby(['manufacturer_id','disbursal_week'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['manufac_week_disbursed_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['manufacturer_id','disbursal_week'],how='left')

####################### State_ID Mean #######################
branch_gps = train_test.groupby(['State_ID','disbursal_week'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['state_week_disbursed_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['State_ID','disbursal_week'],how='left')

####################### branch_id Mean #######################
branch_gps = train_test.groupby(['branch_id','disbursal_week'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['branch_week_disbursed_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['branch_id','disbursal_week'],how='left')

print(train_test.shape)

(345546, 52) ==> (345546, 55)



## aggregation date wise

In [26]:
train_test1 = train_test.copy()
############## Simple aggregation w/o time compomnenet###########
print(train_test.shape, "==>", end=" ")

####################### manufacturer_id Mean #######################
branch_gps = train_test.groupby(['DisbursalDate_new'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['manufac_disbursedDate_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['DisbursalDate_new'],how='left')

####################### State_ID Mean #######################
branch_gps = train_test.groupby(['DisbursalDate_new'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['state_disbursedDate_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['DisbursalDate_new'],how='left')

####################### branch_id Mean #######################
branch_gps = train_test.groupby(['DisbursalDate_new'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['branch_disbursedDate_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['DisbursalDate_new'],how='left')

print(train_test.shape)


(345546, 55) ==> (345546, 58)


In [27]:
############## Aggregation with time compomnenet ###########

print(train_test.shape, "==>", end=" ")

####################### manufacturer_id Mean #######################
branch_gps = train_test.groupby(['manufacturer_id','DisbursalDate_new'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['manufac_date_disbursed_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['manufacturer_id','DisbursalDate_new'],how='left')

####################### State_ID Mean #######################
branch_gps = train_test.groupby(['State_ID','DisbursalDate_new'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['state_date_disbursed_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['State_ID','DisbursalDate_new'],how='left')

####################### branch_id Mean #######################
branch_gps = train_test.groupby(['branch_id','DisbursalDate_new'])[
    'disbursed_amount'].aggregate(['mean'])
branch_gps.columns = ['branch_date_disbursed_mean']
train_test = pd.merge(train_test, branch_gps, 
                      on=['branch_id','DisbursalDate_new'],how='left')

print(train_test.shape)

(345546, 58) ==> (345546, 61)


In [28]:
stat_cols = ['PRI.CURRENT.BALANCE','PRI.DISBURSED.AMOUNT','PRI.SANCTIONED.AMOUNT',
             'PRIMARY.INSTAL.AMT','asset_cost','disbursed_amount']

print(train_test.shape, "==>", end=" ")

train_test['stat_mean']   = train_test[stat_cols].mean(axis=1)
train_test['stat_median'] = train_test[stat_cols].median(axis=1)
train_test['stat_skew']   = train_test[stat_cols].skew(axis=1)
train_test['stat_std']    = train_test[stat_cols].std(axis=1)
print(train_test.shape)

(345546, 61) ==> (345546, 65)


In [29]:
assert train_test.shape[0] == train_df.shape[0]+test_df.shape[0]
print("shape are equal")

shape are equal


In [30]:
branch_gping = train_test.groupby(['DisbursalDate','branch_id','Employee_code_ID'])[
    'disbursed_amount', 'ltv','asset_cost'].mean()
branch_gping.columns = ['date_branch_emp_disbursed_amount', 'date_branch_emp_ltv', 
                        'date_branch_emp_asset_cost']

print(train_test.shape, "==>", end=" ")
train_test = pd.merge(train_test, branch_gping, 
                      on=['DisbursalDate','branch_id','Employee_code_ID'],
                      how='left')
print(train_test.shape)


(345546, 65) ==> (345546, 68)


In [31]:
drop_cols = ['date_of_birth', 'Employee_code_ID','DisbursalDate',
             'CREDIT.HISTORY.LENGTH', 'AVERAGE.ACCT.AGE', 'DisbursalDate_new']

train_test.drop(drop_cols, axis=1, inplace=True)
train_test.head()

,Aadhar_flag,Current_pincode_ID,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,Employment.Type,NO.OF_INQUIRIES,PAN_flag,PRI.CURRENT.BALANCE,PRI.DISBURSED.AMOUNT,PRI.SANCTIONED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.CURRENT.BALANCE,SEC.DISBURSED.AMOUNT,SEC.SANCTIONED.AMOUNT,State_ID,UniqueID,asset_cost,branch_id,disbursed_amount,loan_default,ltv,manufacturer_id,supplier_id,train_flag,disbursal_week,disbursal_day,disbursal_month,age(in years),age(in month),credit_hist_year,credit_hist_month,credit_hist_total_month,loan_tenure_year,loan_tenure_month,loan_tenure_total_month,pri_no_of_accts,no_of_acc,no_of_acc_overdue,Bureau_desc,bureau_score,new_acc_past_month,pay_by_yourself,obtained_amount_per_month,obtained_amount_per_year,manufac_disbursed_mean,state_disbursed_mean,branch_disbursed_mean,manufac_week_disbursed_mean,state_week_disbursed_mean,branch_week_disbursed_mean,manufac_disbursedDate_mean,state_disbursedDate_mean,branch_disbursedDate_mean,manufac_date_disbursed_mean,state_date_disbursed_mean,branch_date_disbursed_mean,stat_mean,stat_median,stat_skew,stat_std,date_branch_emp_disbursed_amount,date_branch_emp_ltv,date_branch_emp_asset_cost
0,1,1441,0,Salaried,0,0,0,0,0,0,0,0,0,6,420825,58400,67,50578,0.0,89.55,45,22807,1,31,3,8,34,34,0,0,0,0,0,0,0,0,0,not_enough_info,0,0,0,0.000000,0.0,49787.567424,52013.951394,52479.194149,46057.862745,49444.791667,50931.649718,51740.371549,51740.371549,51740.371549,46174.107895,49978.261224,50470.271429,18163.000000,0.0,1.001508,28246.510121,50578.0,89.55,58400.0
1,1,1502,1,Self employed,0,0,27600,50200,50200,1991,0,0,0,6,537409,65550,67,47145,1.0,73.23,45,22807,1,39,26,9,33,33,1,11,23,1,11,23,1,1,1,I-Medium Risk,598,0,0,2091.666667,25100.0,49787.567424,52013.951394,52479.194149,50363.817038,52137.326040,52618.855381,55268.452152,55268.452152,55268.452152,50183.237553,51734.682505,52383.056738,40447.666667,48672.5,-1.094696,22401.674818,49761.5,80.81,62925.0
2,1,1497,0,Self employed,0,0,0,0,0,0,0,0,0,6,417566,61360,67,53278,0.0,89.63,45,22807,1,31,1,8,32,32,0,0,0,0,0,0,0,0,0,not_enough_info,0,0,0,0.000000,0.0,49787.567424,52013.951394,52479.194149,46057.862745,49444.791667,50931.649718,51430.433841,51430.433841,51430.433841,45514.829730,48707.741525,51593.740000,19106.333333,0.0,1.000350,29709.537335,53278.0,89.63,61360.0
3,1,1501,0,Self employed,1,0,0,0,0,31,0,0,0,6,624493,66113,67,57513,1.0,88.48,45,22807,1,43,26,10,24,24,1,3,15,0,8,8,3,0,0,L-Very High Risk,305,0,0,0.000000,0.0,49787.567424,52013.951394,52479.194149,51074.109043,52919.600166,53079.192621,56431.085599,56431.085599,56431.085599,51439.126253,53916.178801,54048.882812,20609.500000,15.5,0.999520,32031.752863,57513.0,88.48,66113.0
4,1,1495,0,Self employed,1,0,0,0,0,0,0,0,0,6,539055,60300,67,52378,1.0,88.39,45,22807,1,39,26,9,41,41,0,0,0,0,0,0,0,0,0,not_enough_info,0,0,0,0.000000,0.0,49787.567424,52013.951394,52479.194149,50363.817038,52137.326040,52618.855381,55268.452152,55268.452152,55268.452152,50183.237553,51734.682505,52383.056738,18779.666667,0.0,1.000176,29200.991775,49761.5,80.81,62925.0


In [32]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
col = 'Bureau_desc'
train_test[col] = le.fit_transform(train_test[col])
print(le.classes_)
col = 'Employment.Type'
train_test[col] = le.fit_transform(train_test[col])
print(le.classes_)

['A-Very Low Risk' 'B-Very Low Risk' 'C-Very Low Risk' 'D-Very Low Risk'
 'E-Low Risk' 'F-Low Risk' 'G-Low Risk' 'H-Medium Risk' 'I-Medium Risk'
 'J-High Risk' 'K-High Risk' 'L-Very High Risk' 'M-Very High Risk'
 'not_enough_info']
['Salaried' 'Self employed']


In [33]:
tr_shape = train_df.shape[0]
ts_shape = test_df.shape[0]

del train_df, test_df
gc.collect()

215

In [34]:
train_df = train_test.iloc[:tr_shape, :]
test_df = train_test.iloc[tr_shape:, :]
test_df.reset_index(drop=True, inplace=True)

tr_unique_ids = train_df.UniqueID
ts_unique_ids = test_df.UniqueID

train_df.shape, test_df.shape

((233154, 62), (112392, 62))

In [35]:
target = train_df['loan_default']

train_df.drop(['loan_default','train_flag','UniqueID'], axis=1, inplace=True)
test_df.drop(['loan_default','train_flag','UniqueID'], axis=1, inplace=True)

train_df.shape, test_df.shape

((233154, 59), (112392, 59))

In [36]:
print(train_df.shape, test_df.shape, "==>", end=" ")
train_df.drop(['disbursal_week', 'disbursal_day'], axis=1, inplace=True)
test_df.drop(['disbursal_week', 'disbursal_day'], axis=1, inplace=True)
print(train_df.shape, test_df.shape)


(233154, 59) (112392, 59) ==> (233154, 57) (112392, 57)


In [37]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb


In [38]:
def train_lgb_model(X_train, y_train, X_valid, y_valid, features, param, X_test, num_round):
    """
    Args:
        X_train, X_valid: training and valid data
        y_train, y_valid: training and valid target
        X_test: test-data
        features: training features
    Return:
        oof-pred, test_preds model, model_imp
    """
    _train = lgb.Dataset(X_train[features], label=y_train, feature_name=list(features))
    _valid = lgb.Dataset(X_valid[features], label=y_valid,feature_name=list(features))
    
    clf = lgb.train(param, _train, num_round, 
                    valid_sets = [_train, _valid], 
                    verbose_eval=200, 
                    early_stopping_rounds = 25)                  
    
    oof = clf.predict(X_valid[features], num_iteration=clf.best_iteration)
    test_pred = clf.predict(X_test[features], num_iteration=clf.best_iteration)
    
    lgb_imp = pd.DataFrame(data=[clf.feature_name(), list(clf.feature_importance())]).T
    lgb_imp.columns = ['feature','imp']
    
    return oof, test_pred, clf, lgb_imp
    


In [39]:
def run_cv_lgb(train_df, target, test_df, leaves=None):

    param = {
        'bagging_freq'           : 5,
        'bagging_fraction'       : 0.33,
        'boost_from_average'     : 'false',
        'boost'                  : 'gbdt',
        'feature_fraction'       : 0.3,
        'learning_rate'          : 0.01,
        'max_depth'              : -1,
        'metric'                 : 'auc',
        'min_data_in_leaf'       : 100,
#         'min_sum_hessian_in_leaf': 10.0,
        'num_leaves'             : 30,
        'num_threads'            : 4,
        'tree_learner'           : 'serial',
        'objective'              : 'binary',
        'verbosity'              : 1,
    #     'lambda_l1'              : 0.001,
        'lambda_l2'              : 0.1
    }   
    if leaves is not None:
        param['num_leaves'] = leaves
        print("using leaves: ", param['num_leaves'])

    random_seed = 1234
    n_splits = 3
    num_round = 10000
    feature_imp = pd.DataFrame()
    
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    oof_lgb = np.zeros(len(train_df))
    predictions = np.zeros((len(test_df),n_splits))

    clfs = []
    
    for fold_, (train_index, valid_index) in enumerate(folds.split(train_df, target)):
        print(train_index.shape, valid_index.shape)
        print("Fold {}".format(fold_))
    
        y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
        X_train, X_valid = train_df.iloc[train_index,:], train_df.iloc[valid_index,:]
        features = X_train.columns
        
#         X_train.drop(['disbursal_week','disbursal_day'], axis=1, inplace=True)
#         X_valid.drop(['disbursal_week','disbursal_day'], axis=1, inplace=True)

        num_round = 10000
        oof, test_pred, clf, lgb_imp = train_lgb_model(X_train, y_train, 
                                                       X_valid, y_valid, 
                                                       features, param, 
                                                       test_df, num_round)
        lgb_imp['fold'] = fold_
        feature_imp = pd.concat([feature_imp, lgb_imp], axis=0)
    
        oof_lgb[valid_index] = oof
        predictions[:,fold_] = test_pred
        clfs.append(clf)
        
        score = roc_auc_score(y_valid, oof)
        print( "  auc = ", score )
        print("="*60)
    
    return clfs, feature_imp, oof_lgb, predictions

In [40]:
def make_prediction(file_path, df, test_ids, sub_df):
    """
    Args:
        file_path: file-name with base-path as "submission"
        df: array with shape (test_df.shape[0], cv_fold)
        test_ids: test_ids
        sub_df: submission data-frame
        
    Return:
        output a file with given name
        
    Example: 
    >>> make_prediction(file_path, predictions, ts_unique_ids, sub)
    """
    predictions = np.mean(df, axis=1)
    sub_df = pd.DataFrame({"ID_code":test_ids})
    sub_df["target"] = predictions
    sub_df.columns = sub.columns

    sub_df.to_csv('{}.csv'.format(file_path), index=None)
    print("successfully saved")
#     print(sub_df.shape)
#     print(sub_df.sample(10))


In [41]:
model_lgb1, imp_lgb1, oof_lgb1, pred_lgb1 = run_cv_lgb(train_df, target, test_df)
make_prediction("final-submission", pred_lgb1, ts_unique_ids, sub)


(155435,) (77719,)
Fold 0
Training until validation scores don't improve for 25 rounds.
[200]	training's auc: 0.666546	valid_1's auc: 0.6565
[400]	training's auc: 0.677996	valid_1's auc: 0.663246
[600]	training's auc: 0.687108	valid_1's auc: 0.66718
[800]	training's auc: 0.694494	valid_1's auc: 0.669191
Early stopping, best iteration is:
[953]	training's auc: 0.699553	valid_1's auc: 0.670029
  auc =  0.6700285072977943
(155436,) (77718,)
Fold 1
Training until validation scores don't improve for 25 rounds.
[200]	training's auc: 0.668727	valid_1's auc: 0.652382
[400]	training's auc: 0.679998	valid_1's auc: 0.658444
[600]	training's auc: 0.689012	valid_1's auc: 0.662023
[800]	training's auc: 0.696107	valid_1's auc: 0.66412
[1000]	training's auc: 0.70246	valid_1's auc: 0.665321
[1200]	training's auc: 0.708422	valid_1's auc: 0.666139
Early stopping, best iteration is:
[1304]	training's auc: 0.711343	valid_1's auc: 0.666491
  auc =  0.6664906566135586
(155437,) (77717,)
Fold 2
Training until